<a href="https://colab.research.google.com/github/FrogyKing/MBAIA/blob/main/Principales_tecnologias_de_la_IA/sesion4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea NLP

# Alumno: Luis Gonzalez Alvarez

El archivo "conocimientos.json" que contiene las preguntas y respuestas se encuentra en el repo de Github donde desarrollé la tarea.
https://github.com/FrogyKing/MBAIA/tree/0c2158662c7b6fab8ca3d46886ba72c1cf74ff07/Principales_tecnologias_de_la_IA

In [1]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
with open("conocimientos.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [3]:
preguntas = [item["pregunta"] for item in data]
respuestas = [item["respuesta"] for item in data]

In [4]:
vectorizador = TfidfVectorizer()
X = vectorizador.fit_transform(preguntas)

In [50]:
def responder(pregunta_usuario):
    vector_usuario = vectorizador.transform([pregunta_usuario])
    similitudes = cosine_similarity(vector_usuario, X)
    idx = similitudes.argmax()
 # Umbral de confianza
    if similitudes[0, idx] < 0.2:
        return "Lo siento, solo puedo responder preguntas de Historia del Perú."
    return respuestas[idx]

# P1: Ampliar dataset

El dataset se amplió con 100 preguntas y respuestas en total

In [6]:
len(data)

100

# P2: Probar con preguntas ambiguas o mal escritas

## Sin preprocesamiento

### Ejemplo 1

In [51]:
q_correcta = "Cuando fue la independencia del peru"
q_incorrecta = "Cuando fue la independensia del peru"
print("Versión correcta")
print(responder(q_correcta))
print("Versión incorrecta")
print(responder(q_incorrecta))

Versión correcta
La independencia del Perú fue proclamada por José de San Martín el 28 de julio de 1821.
Versión incorrecta
La esclavitud fue abolida en el Perú el 3 de diciembre de 1854 por el presidente Ramón Castilla.


### Ejemplo 2

In [52]:
q_correcta = "Quien fue bolognesi"
q_incorrecta = "Quien fue bologneci"
print("Versión correcta")
print(responder(q_correcta))
print("Versión incorrecta")
print(responder(q_incorrecta))

Versión correcta
Francisco Bolognesi fue un héroe nacional que defendió Arica en 1880 diciendo: 'Tengo deberes sagrados que cumplir'.
Versión incorrecta
Atahualpa fue el último emperador inca, capturado y ejecutado por los conquistadores españoles en 1533.


Los respuestas son muy sensibles a errores ortograficos debido a que TFidVectorizer trabaja solo con coincidencias exactas de palabras.

Se podría corregir con un corrector ortográfico o usando embedding semánticos

## Con preprocesamiento usando embeddings

In [11]:
%pip install -q --upgrade google-generativeai langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.3 MB/s eta 0:00:00


In [28]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 39.1 MB/s eta 0:00:00


In [29]:
from google.colab import userdata
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema import Document
llm_model = "gemini-2.5-flash"
api_key = userdata.get('GOOGLE_API_KEY')

In [30]:
docs = [
    Document(page_content=d["pregunta"], metadata={"respuesta": d["respuesta"]})
    for d in data
]

In [31]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=api_key)

In [34]:
db = FAISS.from_documents(docs, embeddings)
db.save_local("faiss_historia_peru")

In [54]:
def responder(pregunta_usuario: str) -> str:
    resultados = db.similarity_search(pregunta_usuario, k=1)

    if not resultados:
        return "Lo siento, solo puedo responder preguntas de Historia del Perú."

    mejor_match = resultados[0]
    respuesta = mejor_match.metadata.get("respuesta", "No se encontró una respuesta.")

    return respuesta

### Ejemplo 1

In [55]:
q_correcta = "Cuando fue la independencia del peru"
q_incorrecta = "Cuando fue la independensia del peru"
print("Versión correcta")
print(responder(q_correcta))
print("Versión incorrecta")
print(responder(q_incorrecta))

Versión correcta
José de San Martín proclamó la independencia del Perú el 28 de julio de 1821 en Lima.
Versión incorrecta
José de San Martín proclamó la independencia del Perú el 28 de julio de 1821 en Lima.


### Ejemplo 2

In [56]:
q_correcta = "Quien fue bolognesi"
q_incorrecta = "Quien fue bologneci"
print("Versión correcta")
print(responder(q_correcta))
print("Versión incorrecta")
print(responder(q_incorrecta))

Versión correcta
Francisco Bolognesi fue un héroe nacional que defendió Arica en 1880 diciendo: 'Tengo deberes sagrados que cumplir'.
Versión incorrecta
Francisco Bolognesi fue un héroe nacional que defendió Arica en 1880 diciendo: 'Tengo deberes sagrados que cumplir'.


# P3: Extender el bot a bsuqueda de fuentes de información en la web

In [58]:
%pip install -q langchain-community

In [63]:
%pip install -q -U ddgs

In [60]:
from langchain_community.tools import DuckDuckGoSearchResults

In [98]:
import re

In [64]:
search = DuckDuckGoSearchResults()

In [65]:
def buscar_local(pregunta):
    resultados = db.similarity_search_with_score(pregunta, k=1)
    if not resultados:
        return None, 0.0
    doc, score = resultados[0]
    return doc.metadata.get("respuesta", ""), score

In [99]:
def buscar_web(pregunta):
    buscador = DuckDuckGoSearchResults(max_results=3)
    resultados = buscador.invoke(pregunta)
    texto = resultados[:500]
    enlaces = re.findall(r'https?://[^\s,]+', resultados)
    if not enlaces:
        enlaces = ["https://duckduckgo.com"]
    return texto, enlaces

In [118]:
def responder(pregunta):
    respuesta_local, score = buscar_local(pregunta)

    if score < 0.8:
        _,enlaces = buscar_web(pregunta)
        enlaces_texto = "\n".join([f"{url}" for url in enlaces])
        return f"{respuesta_local}\n\nSe encontró la respuesta en la BD local, pero te muestro algunas referencias web \
        \n\nFuentes consultadas:\n{enlaces_texto}"

    texto_web, enlaces = buscar_web(pregunta)
    enlaces_texto = "\n".join([f"{url}" for url in enlaces])

    return f"{texto_web}\n\nFuentes consultadas:\n{enlaces_texto}"

## Usando la BD local

In [114]:
print(responder("Quien fue francisco pizarro"))

Francisco Pizarro fue el conquistador español que lideró la expedición que derrotó al Imperio Inca en 1532.

Se encontró la respuesta en la BD local, pero te muestro algunas referencias web         

Fuentes consultadas:
https://planlea.edu.do/2025/06/quien-fue-francisco-pizarro/
https://experiencialima.com/quien-fue-francisco-pizarro/
https://estudyando.com/quien-fue-francisco-pizarro/
https://www.thoughtco.com/facts-about-francisco-pizarro-2136550


## Buscando en la web cuando no encuentra similitudes en la BD local

In [119]:
print(responder("Cuando juega el real madrid"))

snippet: Primero que nada, a qué hora juega el Real Madrid depende mucho de la competición en la que estén participando. Por ejemplo, si es La Liga, los partidos suelen ser en horarios diferentes que los de la Champions League., title: A Qué Hora Juega El Real Madrid : Descubre El Horario Exacto Hoy, link: https://madridlive.es/a-que-hora-juega-el-real-madrid-descubre-el-horario-exacto-hoy-2, snippet: El Real Madrid consigue su billete a París en una remontada para la historia. Aunque los goles 

Fuentes consultadas:
https://madridlive.es/a-que-hora-juega-el-real-madrid-descubre-el-horario-exacto-hoy-2
https://cadenaser.com/2022/05/04/el-real-madrid-se-cuela-en-paris-con-una-remontada-para-la-historia/?primarySection=/deportes/futbol
https://www.tiktok.com/discover/qué-tiene-que-pasar-para-el-madrid-y-quedar-entre-los-81
https://as.com/futbol/fotorrelato/el-peor-once-historico-del-real-madrid-f/


# P4: Implementar una interfaz

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets

encabezado_html = """
<div style="
    background: linear-gradient(90deg, #0057B7, #FFD700);
    color: white;
    padding: 15px;
    border-radius: 10px;
    font-family: 'Segoe UI', sans-serif;
    box-shadow: 0 2px 6px rgba(0,0,0,0.2);
">
    <h2 style="margin: 0; text-align: center;">Chatbot de Historia del Perú</h2>
    <p style="margin: 5px 0 0; text-align: center; font-size: 14px;">
        Pregunta sobre hechos históricos, personajes o eventos del Perú.
    </p>
</div>
"""
display(HTML(encabezado_html))

entrada = widgets.Text(
    value='',
    placeholder='Escribe tu pregunta sobre el Perú aca...',
    description='Pregunta:',
    layout=widgets.Layout(width='100%')
)

boton = widgets.Button(
    description='Responder',
    button_style='info',
    tooltip='Haz clic para obtener respuesta'
)

salida = widgets.Output()

# --- Lógica del botón ---
def al_hacer_click(b):
    salida.clear_output()
    pregunta = entrada.value.strip()
    if not pregunta:
        with salida:
            print("Por favor escribe una pregunta.")
        return

    try:
        respuesta = responder(pregunta)
        with salida:
            display(HTML(f"""
            <div style='
                background:#f8f9fa;
                padding:15px;
                border-radius:8px;
                border:1px solid #ddd;
                font-family:Segoe UI;
                line-height:1.5;
            '>
                <b>Respuesta:</b><br><br>
                {respuesta.replace(chr(10), "<br>")}
            </div>
            """))
    except Exception as e:
        with salida:
            print("Oh lo siento, Ocurrió un error:", e)

boton.on_click(al_hacer_click)
display(entrada, boton, salida)